In [2]:
import os
import glob
import altair as alt
import pandas as pd
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
path_base = Path(os.getcwd()).parents[1]
print(path_base)


/notebooks/chicago_bears_nerd


In [16]:
nfl_date = "20221116"
teams = ["Chicago", "Atlanta"]
team_color = "#c83803"

opp_team_color = "#A71930" 
#opp_team_color = "#000000"

In [17]:
file_path = f"{path_base}/data/team_ranks/_nfl_stat*{nfl_date}.csv"


file_paths = glob.glob(os.path.join(file_path))
pds = list()
for file_path in file_paths:
    df = pd.read_csv(file_path)
    pds.append(df)
df = pd.concat(pds)
def_stats = {
    "opponent-points-per-game":.6,
    "opponent-points-per-play":1,
    "opponent-1st-half-points-per-game":1.5,
    "opponent-2nd-half-points-per-game":1.6,
    "opponent-rushing-touchdowns-per-game":1.7,
    "opponent-passing-touchdowns-per-game":1.8,
    "opponent-yards-per-game":1.85,
    "opponent-yards-per-play":1.9,
    "opponent-rushing-yards-per-game":2,
    "opponent-yards-per-rush-attempt":2.1,
    "opponent-passing-yards-per-game":2.6,
    "opponent-yards-per-pass-attempt":3,
    "defensive-points-per-game":4,
    "sacks-per-game":10,
    "interceptions-per-game":11,
    "opponent-fumbles-per-game":12,
    }


stats = {
    "points-per-game":2,
    "points-per-play":3.5,
    "1st-half-points-per-game":3.9,
    "2nd-half-points-per-game":4,
    "rushing-touchdowns-per-game":4.1,
    "passing-touchdowns-per-game":4.2,
    "yards-per-game":4.5,
    "yards-per-play":4.8,
    "rushing-yards-per-game":4.9,
    "yards-per-rush-attempt":5,
    "rushing-play-pct":5.5,
    "passing-yards-per-game":5.6,
    "yards-per-pass-attempt":6,
    "passing-play-pct":7,
    "qb-sacked-per-game":10,
    "interceptions-thrown-per-game":11,
    "fumble-per-game":12,
    "penalties-per-game":13
    }

df["_stat"] = df.url.str.split("/", expand=True)[3]
df["Stat"] = df._stat.str.split("-").str.join(" ").str.title()
df["Stat"] = df.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")

In [87]:
df

Stat     Team  value
0    2Nd Quarter Time Of Possession Share Pct  Chicago      9
1    4Th Quarter Time Of Possession Share Pct  Chicago     15
2                              Completion Pct  Chicago     29
3                              Completions Pg  Chicago     32
4                  Fourth Down Conversions Pg  Chicago     20
..                                        ...      ...    ...
115                                  Sack Pct  Atlanta     31
116                                  Sacks Pg  Atlanta     31
117                          Seconds Per Play  Atlanta     30
118                            Third Downs Pg  Atlanta     32
119                                  Yards Pg  Atlanta     27

[120 rows x 3 columns]

In [19]:
df = df[df.Team.isin(teams)]
df_piv = df.pivot_table(index="Stat", columns="Team", values="Rank")
df_piv = df_piv[df_piv[teams[1]] >= 25]
df = df_piv.reset_index().melt(value_vars=teams, id_vars="Stat")

In [20]:
y_cfgs = {"shorthand":"Team:O", "title": ""}
x_cfgs = {"shorthand":"value:Q", "title":"Rank"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
axisLeftKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)
axisBottomKwargs = dict(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)
legendKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10, cornerRadius=10, labelFontSize=15, titleFontSize=15)

In [86]:
bars = alt.Chart(df, height=200, width=160)\
.mark_bar()\
.encode(
    x=alt.X(**y_cfgs, axis=None),
    y=alt.Y(**x_cfgs),
    color=(alt.Color("Team", scale=alt.Scale(domain=teams, range=[team_color, opp_team_color]))),
    tooltip=[
        alt.Tooltip("Stat", title="Stat"),
                alt.Tooltip("Team", title="Team"),
                        alt.Tooltip("value", title="Rank"),
            ]
)
text = bars.mark_text(
    align='left',
    baseline='middle',
    #dx=-10,  # Nudges text to right so it doesn't appear on top of the bar
    dy=-7,
    fontSize = 17
).encode(
    text='value:Q'
)

plot = (bars + text).facet(facet=alt.Facet("Stat:O", title="", header=alt.Header(labelColor="white", labelFontSize=17,)), columns=3, title=f"Statistical Categories Where {teams[1]} Ranks >= 25").configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(**axisLeftKwargs)\
.configure_axisBottom(**axisBottomKwargs)\
.configure_title(color=secondary_color, fontSize=25)\
.configure_legend(**legendKwargs)
plot.save("chart_json/bad_rank.json")
plot

alt.FacetChart(...)